In [4]:
import pickle
import torch
from torch import device, cuda
from environs import Env
from sentence_transformers import SentenceTransformer, util
from fastapi import FastAPI, Form, Request
from fastapi.responses import JSONResponse
from fastapi.encoders import jsonable_encoder
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import JSONResponse
from pydantic import BaseModel
from typing import Optional, List
import uvicorn

In [5]:
env = Env()
env.read_env()

embeddings = env.str("EMBEDDINGS", "pickle\\embeddings.pt")
sentences = env.str("SENTENCES", "pickle\\sentences.pkl")
paragraphs = env.str("PARAGRAPHS", "pickle\\paragraphs.pkl")
documents = env.str("DOCUMENTS", "pickle\\documents.pkl")

sentence_transformers_model = env.str(
    "SENTENCE_TRANSFORMERS_MODEL",
    "eduardofv/stsb-m-mt-es-distiluse-base-multilingual-cased-v1",
)

In [6]:
def read_pickle(pickle_file: str, object_type: str):
    """
    Read a pickle file given
    """
    with open(pickle_file, "rb") as fich:
        if object_type == "tensor":
            read_file = torch.load(fich, map_location='cpu')
        elif object_type == "list":
            read_file = pickle.load(fich)
        else:
            raise("Error: No available object type")
    return read_file


In [7]:
app = FastAPI()
CORS_ORIGINS = ["*"]
app.add_middleware(
    CORSMiddleware,
    allow_origins=CORS_ORIGINS,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)
device = device('cuda' if cuda.is_available() else 'cpu')
print(f"Device selected: {device}")

Device selected: cpu


In [10]:
all_embedding_from_source = read_pickle(embeddings, "tensor")
all_sentences_from_source = read_pickle(sentences, "list")
all_documents_from_source = read_pickle(documents, "list")
all_paragraphs_from_source = read_pickle(paragraphs, "list")

In [11]:
model = SentenceTransformer(sentence_transformers_model)

In [17]:
class Search(BaseModel):
    query: List[str]
    source: str

In [18]:
@app.get("/")
def root():
    return {"message": "Hello"}

In [19]:
@app.post("/search")
def compute_input(search : Search) -> dict:
    """
    - Convert the input query to embedding format.
    - Calculates the cosine distance.
    - Get the highest scores ordered from highest to lowest.
    """
    input_embeddings = model.encode(search.query, convert_to_tensor=True)
    cosine_scores = util.pytorch_cos_sim(input_embeddings, all_embedding_from_source)
    best = torch.topk(cosine_scores, 5)
    output = []
    # best[0]-> Score de los topk resultados
    # best[1]-> Índice de los topk resultados (respecto al corpus)
    for score, idx in zip(best[0], best[1]):
        for j, i in enumerate(idx):  
            output.append({"sentence":all_sentences_from_source[i], "score":round(score[j].item(),4)})
            output.append({"sentence":all_sentences_from_source[i], "score":round(score[j].item(),4), "document":all_documents_from_source[i]})
    print(output)
    return JSONResponse(content = output)

In [20]:
uvicorn.run(app, host="0.0.0.0", port=8000)

RuntimeError: asyncio.run() cannot be called from a running event loop